In [1]:
print("Notebook funcionando")

Notebook funcionando


In [ ]:
# Instalar librerias
import pandas as pd
import requests
import os
from dotenv import load_dotenv

print("Librerias cargadas")

Librerias cargadas


In [27]:
# Establecer directorio de trabajo en la raíz del proyecto
os.chdir("/Users/javiermondragon/Documents/data_projects/inflation-predictor")

In [26]:
# Cargar variables de entorno
load_dotenv()

True

In [28]:
api_key = os.getenv("FRED_API_KEY")
print(f"API Key cargada: {api_key[:5]}...")

API Key cargada: b1ab9...


In [35]:
# Probar cada serie una por una
series_a_probar = ["CPIAUCSL", "FEDFUNDS", "DCOILWTICO", "NASDAQQGLDI"]

for serie in series_a_probar:
    params = {
        "series_id": serie,
        "api_key": api_key,
        "file_type": "json"
    }
    response = requests.get(url, params=params)
    data = response.json()
    
    print(f"\n{serie}:")
    print(f"  Status: {response.status_code}")
    print(f"  Claves: {data.keys()}")
    
    if "observations" in data:
        print(f"  Registros: {len(data['observations'])}")
    else:
        print(f"  ERROR: {data.get('error_message', 'Sin mensaje')}")


CPIAUCSL:
  Status: 200
  Claves: dict_keys(['realtime_start', 'realtime_end', 'observation_start', 'observation_end', 'units', 'output_type', 'file_type', 'order_by', 'sort_order', 'count', 'offset', 'limit', 'observations'])
  Registros: 949

FEDFUNDS:
  Status: 200
  Claves: dict_keys(['realtime_start', 'realtime_end', 'observation_start', 'observation_end', 'units', 'output_type', 'file_type', 'order_by', 'sort_order', 'count', 'offset', 'limit', 'observations'])
  Registros: 859

DCOILWTICO:
  Status: 200
  Claves: dict_keys(['realtime_start', 'realtime_end', 'observation_start', 'observation_end', 'units', 'output_type', 'file_type', 'order_by', 'sort_order', 'count', 'offset', 'limit', 'observations'])
  Registros: 10469

NASDAQQGLDI:
  Status: 200
  Claves: dict_keys(['realtime_start', 'realtime_end', 'observation_start', 'observation_end', 'units', 'output_type', 'file_type', 'order_by', 'sort_order', 'count', 'offset', 'limit', 'observations'])
  Registros: 3468


In [36]:
# ============================================
# SERIES CORREGIDASA UTILIZAR
# ============================================

series = {
    "CPIAUCSL": "cpi",           # Inflación (lo que predecimos)
    "FEDFUNDS": "fed_rate",      # Tasa de interés
    "DCOILWTICO": "oil_price",   # Precio petróleo
    "NASDAQQGLDI": "gold_price"  # Precio oro (código corregido)
}

# ============================================
# DESCARGAR Y GUARDAR DATOS CRUDOS
# ============================================

for codigo, nombre in series.items():
    print(f"Descargando {codigo}...", end=" ")
    
    params = {
        "series_id": codigo,
        "api_key": api_key,
        "file_type": "json"
    }
    
    response = requests.get(url, params=params)
    data = response.json()
    
    df = pd.DataFrame(data["observations"])
    df.to_csv(f"data/raw/{nombre}_raw.csv", index=False)
    
    print(f"OK - {len(df)} filas")

print("\n✓ Todos los archivos guardados en data/raw/")

Descargando CPIAUCSL... OK - 949 filas
Descargando FEDFUNDS... OK - 859 filas
Descargando DCOILWTICO... OK - 10469 filas
Descargando NASDAQQGLDI... OK - 3468 filas

✓ Todos los archivos guardados en data/raw/


In [37]:
# ============================================
# PASO 3: PERFILAR DATOS
# ============================================

for nombre in ["cpi", "fed_rate", "oil_price", "gold_price"]:
    df = pd.read_csv(f"data/raw/{nombre}_raw.csv")
    
    print(f"\n{'='*50}")
    print(f"{nombre.upper()}")
    print(f"{'='*50}")
    print(f"Filas: {len(df)}")
    print(f"Columnas: {list(df.columns)}")
    print(f"Tipos: {df.dtypes.to_dict()}")
    print(f"Fecha mínima: {df['date'].min()}")
    print(f"Fecha máxima: {df['date'].max()}")
    print(f"Valores nulos en 'value': {df['value'].isna().sum()}")
    print(f"Valores únicos raros en 'value': {df[df['value'] == '.']['value'].count()}")


CPI
Filas: 949
Columnas: ['realtime_start', 'realtime_end', 'date', 'value']
Tipos: {'realtime_start': dtype('O'), 'realtime_end': dtype('O'), 'date': dtype('O'), 'value': dtype('O')}
Fecha mínima: 1947-01-01
Fecha máxima: 2026-01-01
Valores nulos en 'value': 0
Valores únicos raros en 'value': 1

FED_RATE
Filas: 859
Columnas: ['realtime_start', 'realtime_end', 'date', 'value']
Tipos: {'realtime_start': dtype('O'), 'realtime_end': dtype('O'), 'date': dtype('O'), 'value': dtype('float64')}
Fecha mínima: 1954-07-01
Fecha máxima: 2026-01-01
Valores nulos en 'value': 0
Valores únicos raros en 'value': 0

OIL_PRICE
Filas: 10469
Columnas: ['realtime_start', 'realtime_end', 'date', 'value']
Tipos: {'realtime_start': dtype('O'), 'realtime_end': dtype('O'), 'date': dtype('O'), 'value': dtype('O')}
Fecha mínima: 1986-01-02
Fecha máxima: 2026-02-17
Valores nulos en 'value': 0
Valores únicos raros en 'value': 369

GOLD_PRICE
Filas: 3468
Columnas: ['realtime_start', 'realtime_end', 'date', 'value']

## Problemas encontrados en los datos crudos

1. **Valores faltantes:** CPI tiene 1 valor ".", OIL_PRICE tiene 369, GOLD_PRICE tiene 121
2. **Frecuencias:** CPI y FED_RATE son mensuales. OIL y GOLD son diarias.
3. **Rango de fechas:** GOLD solo tiene datos desde 2012-11. Ese será nuestro límite inferior.
4. **Tipos:** Todas las columnas son texto (object). Hay que convertir.

In [41]:
# ============================================
# PASO 4: IDENTIFICAR PROBLEMAS EN CADA SERIE
# ============================================

archivos = {
    "cpi": "data/raw/cpi_raw.csv",
    "fed_rate": "data/raw/fed_rate_raw.csv",
    "oil_price": "data/raw/oil_price_raw.csv",
    "gold_price": "data/raw/gold_price_raw.csv"
}

for nombre, ruta in archivos.items():
    df = pd.read_csv(ruta)
    
    print(f"\n{'='*50}")
    print(f"{nombre.upper()}")
    print(f"{'='*50}")
    
    # 1. Valores no convertibles a número
    numerico = pd.to_numeric(df["value"], errors="coerce")
    no_numericos = df[numerico.isna()]["value"].unique()
    print(f"1. Valores no numéricos: {no_numericos}")
    
    # 2. Valores nulos originales
    print(f"2. Nulos originales: {df['value'].isna().sum()}")
    
    # 3. Duplicados
    print(f"3. Filas duplicadas: {df.duplicated().sum()}")
    
    # 4. Rango de fechas
    print(f"4. Fechas: {df['date'].min()} a {df['date'].max()}")


CPI
1. Valores no numéricos: ['.']
2. Nulos originales: 0
3. Filas duplicadas: 0
4. Fechas: 1947-01-01 a 2026-01-01

FED_RATE
1. Valores no numéricos: []
2. Nulos originales: 0
3. Filas duplicadas: 0
4. Fechas: 1954-07-01 a 2026-01-01

OIL_PRICE
1. Valores no numéricos: ['.']
2. Nulos originales: 0
3. Filas duplicadas: 0
4. Fechas: 1986-01-02 a 2026-02-17

GOLD_PRICE
1. Valores no numéricos: ['.']
2. Nulos originales: 0
3. Filas duplicadas: 0
4. Fechas: 2012-11-05 a 2026-02-18


## Reglas de limpieza

1. Reemplazar "." por NaN
2. Quedarme solo con date y value
3. Convertir date a fecha, value a número
4. Renombrar value según la serie

In [42]:
# ============================================
# PASO 5: APLICAR LIMPIEZA A TODAS LAS SERIES
# ============================================

def limpiar_serie(ruta, nombre_columna):
    df = pd.read_csv(ruta)
    df = df[["date", "value"]]
    df["value"] = df["value"].replace(".", pd.NA)
    df["date"] = pd.to_datetime(df["date"])
    df["value"] = pd.to_numeric(df["value"])
    df = df.rename(columns={"value": nombre_columna})
    return df

df_cpi = limpiar_serie("data/raw/cpi_raw.csv", "cpi")
df_fed = limpiar_serie("data/raw/fed_rate_raw.csv", "fed_rate")
df_oil = limpiar_serie("data/raw/oil_price_raw.csv", "oil_price")
df_gold = limpiar_serie("data/raw/gold_price_raw.csv", "gold_price")

print("Series limpiadas:")
print(f"  CPI: {len(df_cpi)} filas, tipo: {df_cpi['cpi'].dtype}")
print(f"  FED: {len(df_fed)} filas, tipo: {df_fed['fed_rate'].dtype}")
print(f"  OIL: {len(df_oil)} filas, tipo: {df_oil['oil_price'].dtype}")
print(f"  GOLD: {len(df_gold)} filas, tipo: {df_gold['gold_price'].dtype}")

Series limpiadas:
  CPI: 949 filas, tipo: float64
  FED: 859 filas, tipo: float64
  OIL: 10469 filas, tipo: float64
  GOLD: 3468 filas, tipo: float64


In [43]:
# ============================================
# PASO 6: CONVERTIR SERIES DIARIAS A MENSUALES
# ============================================

# OIL: promedio mensual
df_oil_mensual = df_oil.set_index("date").resample("MS").mean().reset_index()

# GOLD: promedio mensual
df_gold_mensual = df_gold.set_index("date").resample("MS").mean().reset_index()

print("Conversión a mensual:")
print(f"  OIL: {len(df_oil)} diarias → {len(df_oil_mensual)} mensuales")
print(f"  GOLD: {len(df_gold)} diarias → {len(df_gold_mensual)} mensuales")

Conversión a mensual:
  OIL: 10469 diarias → 482 mensuales
  GOLD: 3468 diarias → 160 mensuales


In [44]:
# ============================================
# PASO 7: UNIR TODAS LAS SERIES
# ============================================

# Empezar con CPI
df_final = df_cpi.copy()

# Unir FED
df_final = df_final.merge(df_fed, on="date", how="outer")

# Unir OIL (mensual)
df_final = df_final.merge(df_oil_mensual, on="date", how="outer")

# Unir GOLD (mensual)
df_final = df_final.merge(df_gold_mensual, on="date", how="outer")

# Ordenar por fecha
df_final = df_final.sort_values("date").reset_index(drop=True)

print(f"Dataset unido: {len(df_final)} filas")
print(f"Columnas: {list(df_final.columns)}")
print(f"Rango: {df_final['date'].min()} a {df_final['date'].max()}")
print(f"\nPrimeras filas:")
df_final.head()

Dataset unido: 950 filas
Columnas: ['date', 'cpi', 'fed_rate', 'oil_price', 'gold_price']
Rango: 1947-01-01 00:00:00 a 2026-02-01 00:00:00

Primeras filas:


,date,cpi,fed_rate,oil_price,gold_price
0,1947-01-01,21.48,NaN,NaN,NaN
1,1947-02-01,21.62,NaN,NaN,NaN
2,1947-03-01,22.00,NaN,NaN,NaN
3,1947-04-01,22.00,NaN,NaN,NaN
4,1947-05-01,21.95,NaN,NaN,NaN


In [45]:
# Ver cuántos NaN hay por columna
print("Valores nulos por columna:")
print(df_final.isna().sum())

# Ver desde cuándo hay datos completos
df_completo = df_final.dropna()
print(f"\nFilas con datos completos: {len(df_completo)}")
print(f"Desde: {df_completo['date'].min()}")
print(f"Hasta: {df_completo['date'].max()}")

Valores nulos por columna:
date            0
cpi             2
fed_rate       91
oil_price     468
gold_price    790
dtype: int64

Filas con datos completos: 158
Desde: 2012-11-01 00:00:00
Hasta: 2026-01-01 00:00:00


In [46]:
# ============================================
# PASO 8: QUEDARSE SOLO CON DATOS COMPLETOS
# ============================================

df_modelo = df_final.dropna().reset_index(drop=True)

print(f"Dataset para modelo: {len(df_modelo)} filas")
print(f"Desde: {df_modelo['date'].min()}")
print(f"Hasta: {df_modelo['date'].max()}")
print(f"\nNulos restantes: {df_modelo.isna().sum().sum()}")
df_modelo.head()

Dataset para modelo: 158 filas
Desde: 2012-11-01 00:00:00
Hasta: 2026-01-01 00:00:00

Nulos restantes: 0


,date,cpi,fed_rate,oil_price,gold_price
0,2012-11-01,231.249,0.16,86.531429,6914.752632
1,2012-12-01,231.221,0.16,87.859500,6763.126842
2,2013-01-01,231.679,0.14,94.756667,6654.364286
3,2013-02-01,232.937,0.15,95.308947,6488.243500
4,2013-03-01,232.282,0.14,92.938500,6327.899500


In [47]:
# ============================================
# PASO 9: GUARDAR DATOS PROCESADOS
# ============================================

df_modelo.to_csv("data/processed/dataset_modelo.csv", index=False)

print("Guardado: data/processed/dataset_modelo.csv")
print(f"Filas: {len(df_modelo)}")
print(f"Columnas: {list(df_modelo.columns)}")

Guardado: data/processed/dataset_modelo.csv
Filas: 158
Columnas: ['date', 'cpi', 'fed_rate', 'oil_price', 'gold_price']
